# Preparação do ambiente

## Bibliotecas

In [1]:
import pandas as pd
import re
import warnings

from datetime import datetime
from datetime import timedelta

from pathlib import Path

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

## Constantes e funções auxiliares

In [10]:
ISSUES_FOLDER = Path(r'../datasets/inspecao_ecommerce/issues')
FILE_CLOSED_ISSUES = '../datasets/inspecao_ecommerce/avaliacao_regulatron/closed_issues.csv'
FILE_ANUNCIOS_REGULATRON = '../datasets/inspecao_ecommerce/avaliacao_regulatron/Anuncios.xlsx'

# Carga e prepação dos dados

## Tarefas de rotulagem (segunda fiscalização)

In [4]:
files_issues = [file for file in ISSUES_FOLDER.rglob('*.csv')]

df_issues = pd.concat([pd.read_csv(file,sep=';',quotechar="'") for file in files_issues])
df_issues['Qual foi a data da coleta?'] = pd.to_datetime(df_issues['Qual foi a data da coleta?'],format='mixed')
df_issues['Qual foi a data da coleta?'] = df_issues['Qual foi a data da coleta?'].dt.date
df_issues['Arquivo PDF'] = df_issues['Descrição'].apply(
    lambda descricao: re.search('https.+pdf',descricao).group() 
    if re.search('https.+pdf',descricao) 
    else None
)
df_issues['nome'] = df_issues['Descrição'].apply(lambda descricao: descricao[3:].split('\n')[0])

df_issues.head()

,url,Projeto,Tipo,Situação,Título,Descrição,Data de início,Data limite,Atribuído para,Unidade de Lotação,...,Valor,Existe campo código SCH?,Código SCH foi fornecido?,Qual código SCH fornecido?,Código SCH é o do produto?,Código EAN foi fornecido?,Qual código EAN fornecido?,Código EAN é o do produto?,Arquivo PDF,nome
0,https://www.amazon.com.br/Smartphone-Infinix-B...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000001,"h1. Smartphone Infinix Hot 30i, 8GB RAM 128GB,...",2024-07-10,17/07/2024,werther,GR03,...,998.0,Sim,Sim,5.136232e+10,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,"Smartphone Infinix Hot 30i, 8GB RAM 128GB, Câ..."
1,https://www.amazon.com.br/Samsung-Galaxy-Proce...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000002,h1. Samsung Galaxy S23 5G 128GB Tela 6.1'' 8GB...,2024-07-10,17/07/2024,yoshida,GR03,...,3297.0,Sim,Sim,1.715222e+11,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,Samsung Galaxy S23 5G 128GB Tela 6.1'' 8GB RA...
2,https://www.amazon.com.br/Celular-Xiaomi-Redmi...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000003,h1. Celular Xiaomi Redmi 12C 128GB/4GB de RAM/...,2024-07-10,17/07/2024,werther,GR03,...,772.0,Sim,Sim,2.176222e+11,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,Celular Xiaomi Redmi 12C 128GB/4GB de RAM/Dua...
3,https://www.amazon.com.br/BLU-Smartphone-B9-Br...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000004,h1. BLU Smartphone B9 128GB 4GB RAM Bronze 4G ...,2024-07-10,17/07/2024,yoshida,GR03,...,749.0,Sim,Sim,1.301322e+11,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,BLU Smartphone B9 128GB 4GB RAM Bronze 4G Tel...
4,https://www.amazon.com.br/Samsung-Galaxy-Câmer...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240710_000005,h1. Samsung Galaxy A04s 64GB 4G Wi-Fi Tela 6.5...,2024-07-10,17/07/2024,werther,GR03,...,1139.0,Sim,Sim,7.759220e+10,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,Samsung Galaxy A04s 64GB 4G Wi-Fi Tela 6.5'' ...


## Resultados da segunda fiscalização

In [7]:

df_resultados = pd.read_csv(FILE_CLOSED_ISSUES,sep=';')
df_resultados = df_resultados[
    (df_resultados['Marketplace']=='Amazon')
    &(df_resultados['Texto da busca']=='smartphone')
]

columns_to_keep = ['Título', 'url', 'Arquivo PDF']
# df_issues[df_issues['Marketplace']=='Amazon'][columns_to_keep].to_excel('../datasets/inspecao_ecommerce/avaliacao_regulatron/issues_screenshots.xlsx')
df_resultados = df_resultados.merge(
    df_issues[df_issues['Marketplace']=='Amazon'][columns_to_keep]
)
# estabelecer grupo por período de coleta
df_resultados['coleta'] = '2024-07-08'
df_resultados['nome'] = ''

columns_to_keep = ['url', 'Arquivo PDF', 'coleta', 'nome', 'Qual o tipo do produto? ', 'Fabricante', 'Modelo do Produto', 'Qual código SCH fornecido?', 'Qual código EAN fornecido?']
df_resultados = df_resultados[columns_to_keep]

new_columns = ['url', 'screenshot', 'coleta', 'nome', 'tipo_subcategoria', 'fabricante', 'modelo', 'certificado', 'ean_gtin']
df_resultados.columns = new_columns

df_resultados

,url,screenshot,coleta,nome,tipo_subcategoria,fabricante,modelo,certificado,ean_gtin
0,https://www.amazon.com.br/Samsung-Galaxy-Câmer...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Samsung,SM-A145MLGGZTO | galaxy,208152200953,0
1,https://www.amazon.com.br/Smartphone-Zenfone-S...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Asus,Zenfone 10,118332303109,0
2,https://www.amazon.com.br/Samsung-Galaxy-Bater...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Samsung,WMN5090AE,095832300953,0
3,https://www.amazon.com.br/Casos-Volta-Ultra-fi...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Outros,F2,case,0,0
4,https://www.amazon.com.br/Samsung-Celular-Gala...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Samsung,A35,235482300953,0
...,...,...,...,...,...,...,...,...,...
63,https://www.amazon.com.br/Samsung-Galaxy-Câmer...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Samsung,A04s,077592200953,0
64,https://www.amazon.com.br/BLU-Smartphone-B9-Br...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Blu,B9,130132206887,0
65,https://www.amazon.com.br/Celular-Xiaomi-Redmi...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Xiaomi,MZB0DH2EU |,217622209185,0
66,https://www.amazon.com.br/Samsung-Galaxy-Proce...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Samsung,SM-S911BZKJZTO | S23,171522200953,0


## Raspagem do Regulatron

In [11]:
%%time
dtype = {
    'certificado': 'str',
    'ean_gtin': 'str'
}

# importar resultados de raspagem
df_anuncios_regulatron = pd.read_excel(FILE_ANUNCIOS_REGULATRON,dtype=dtype)

# manter apenas Amazon
df_anuncios_regulatron = df_anuncios_regulatron[df_anuncios_regulatron['marketplace']=='Amazon']
# manter apenas busca por smartphone
df_anuncios_regulatron = df_anuncios_regulatron[df_anuncios_regulatron['palavra_busca'].str.contains('smartphone')] 

df_anuncios_regulatron['certificado'] = df_anuncios_regulatron['certificado'].fillna('0')
df_anuncios_regulatron['ean_gtin'] = df_anuncios_regulatron['ean_gtin'].fillna('0')

# agrupar coletas
map_coletas = {
    '2024-10-02': '2024-10-02', 
    '2024-10-03': '2024-10-02',
    '2024-10-04': '2024-10-02',
    '2024-10-10': '2024-10-10',
    '2024-10-11': '2024-10-10',
    '2024-10-22': '2024-10-22',
}
df_anuncios_regulatron['coleta'] = df_anuncios_regulatron.data.apply(lambda data: data[:10]).map(map_coletas)


# # manter apenas urls que estavam na primeira fiscalização (julho)
# df_anuncios_regulatron = df_anuncios_regulatron.merge(df_espatula.url)
# # contar quantas vezes cada url aparece
# df_anuncios_regulatron['url_count'] = df_anuncios_regulatron[['url','nome']].groupby('url').transform('count')

columns_to_keep = ['url', 'screenshot', 'coleta', 'nome', 'subcategoria', 'fabricante', 'modelo', 'certificado', 'ean_gtin']
df_anuncios_regulatron = df_anuncios_regulatron[columns_to_keep]

df_anuncios_regulatron.columns = new_columns
df_anuncios_regulatron

CPU times: total: 1.25 s
Wall time: 1.25 s


,url,screenshot,coleta,nome,tipo_subcategoria,fabricante,modelo,certificado,ean_gtin
4,https://www.amazon.com.br/Smartphone-Tecno-Cam...,a431d49c-81f7-47f8-bd5f-5b56d95c9271.pdf,2024-10-02,"Smartphone Tecno Camon 19 Neo CH6i, 6/128 GB, ...",Celulares e Smartphones,Tecno,CH6i,112062213713,0
5,https://www.amazon.com.br/Smartphone-Samsung-G...,ec9bf59f-7c44-4448-8845-585c80b97662.pdf,2024-10-02,"Smartphone Samsung Galaxy S24 Ultra, Galaxy AI...",Celulares e Smartphones,Samsung,SM-S928B/DS,206372300953,0
6,https://www.amazon.com.br/Smartphone-Xiaomi-Re...,f12017ca-7ef3-4ef3-9c2e-7fd40b24c97f.pdf,2024-10-02,Smartphone Xiaomi Redmi Note 13 5G 256GB / 8GB...,Celulares e Smartphones,Xiaomi,2312DRAABG | Redmi Note 13 5G | 2312DRAABG,175602314550,0
7,https://www.amazon.com.br/Apple-iPhone-15-256-...,00869d36-0668-4f14-865f-20a0ce3b1bf2.pdf,2024-10-02,Apple iPhone 15 (256 GB) — Azul,Celulares e Smartphones,Apple,NaN,127572301993,0
17,https://www.amazon.com.br/Smartphone-Xiaomi-Re...,b81b4402-bbc1-4403-a843-1f1f84587c62.pdf,2024-10-02,"Smartphone Xiaomi Redmi 13C 128GB Tela 6.74"" 4...",Celulares e Smartphones,Xiaomi,2310FPCA4G | POCO C65 | 6X86,0,0
...,...,...,...,...,...,...,...,...,...
2920,https://www.amazon.com.br/Cabo-Toocki-Classic-...,4d5b1779-11fa-4b80-896e-38c6d2237689.pdf,2024-10-22,Cabo Toocki Classic USB A para C com Carregame...,Cabos USB,Generic,TXCT-SJ03-Z,0,0
2921,https://www.amazon.com.br/Cabo-Toocki-Mont-Car...,0934d350-7bd5-42fb-8da9-261915578630.pdf,2024-10-22,Cabo Toocki Mont USB C 100W 6A com Carregament...,Cabos USB,Genérico,TXCTT12-HY01,0,0
2922,https://www.amazon.com.br/Cabo-Toocki-Classic-...,6e3c84c9-d604-46a0-ba72-7a90f104cdbc.pdf,2024-10-22,Cabo Toocki Classic USB C com Carregamento Ráp...,Cabos USB,Generic,TXCTT1A-SJ03-Z,0,0
2923,https://www.amazon.com.br/Toocki-Bluish-Carreg...,3b65f47d-5d47-46de-931e-3ddc6a36004b.pdf,2024-10-22,Cabo Toocki Bluish USB C com Carregamento Supe...,Cabos USB,Genérico,Bluish C-C 60W,0,0


# Anúncios comuns

In [13]:
# unir os dataframes
df_anuncios_analise = pd.concat([df_resultados, df_anuncios_regulatron])
df_anuncios_analise.coleta = pd.to_datetime(df_anuncios_analise.coleta)
df_anuncios_analise.modelo = df_anuncios_analise.modelo.fillna('0')

sort_keys = ['url', 'coleta']
df_anuncios_analise = df_anuncios_analise.sort_values(by=sort_keys)
df_anuncios_analise = df_anuncios_analise.drop_duplicates(subset=sort_keys)

df_anuncios_analise['certificado_anuncio'] = df_anuncios_analise['certificado']!='0'
df_anuncios_analise['certificado_anuncio'] = df_anuncios_analise['certificado_anuncio'].astype(int)

df_anuncios_analise['ean_gtin_anuncio'] = df_anuncios_analise['ean_gtin']!='0'
df_anuncios_analise['ean_gtin_anuncio'] = df_anuncios_analise['ean_gtin_anuncio'].astype(int)

# df_anuncios_analise = df_anuncios_analise.drop_duplicates(subset='url')
# df_anuncios_analise = df_anuncios_analise.reset_index(drop=True)
# df_anuncios_analise.to_excel('../datasets/inspecao_ecommerce/avaliacao_regulatron/anuncios_analise_historico.xlsx')

df_anuncios_analise['ocorrencias'] = df_anuncios_analise[['url','modelo']].groupby('url').transform('count')
# df_anuncios_analise = df_anuncios_analise[df_anuncios_analise['ocorrencias']>2]

df_anuncios_analise = df_anuncios_analise.reset_index(drop=True)
df_anuncios_analise.to_excel('../datasets/inspecao_ecommerce/avaliacao_regulatron/anuncios_para_limpar_2.xlsx')
df_anuncios_analise

,url,screenshot,coleta,nome,tipo_subcategoria,fabricante,modelo,certificado,ean_gtin,certificado_anuncio,ean_gtin_anuncio,ocorrencias
0,https://www.amazon.com.br/5000mAh-display-Came...,ca5c984c-5cfc-45d3-b493-757ceb6796d3.pdf,2024-10-02,"POCO C65 6GB+128GB 5000mAh 6.74"" 90Hz HD+ disp...",Celulares e Smartphones,Xiaomi,2022 | 2310FPCA4G | POCO C65,0,0,0,0,2
1,https://www.amazon.com.br/5000mAh-display-Came...,3767f0e4-1026-40ca-b598-c481059fc6ec.pdf,2024-10-22,"POCO C65 6GB+128GB 5000mAh 6.74"" 90Hz HD+ disp...",Celulares e Smartphones,Xiaomi,0,175342314550,0,1,0,2
2,https://www.amazon.com.br/5000mAh-display-Came...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,,Celulares e Smartphones,Xiaomi,POCO C65,175342314550,0,1,0,1
3,https://www.amazon.com.br/Apple-iPhone-12-128-...,1b1a94fb-db78-44d6-9bdb-c6c7e01eb1c2.pdf,2024-10-10,Apple iPhone 12 (128 GB) - Preto,Celulares e Smartphones,Apple,0,129962001993,0,1,0,1
4,https://www.amazon.com.br/Apple-iPhone-13-128-...,d6c099d3-7941-4faa-8910-181026dbda60.pdf,2024-10-02,Apple iPhone 13 (128 GB) - Meia-noite,Celulares e Smartphones,Apple,0,112722101993,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
732,https://www.amazon.com.br/Yamaha-93306-30605-0...,154c1059-db5e-4d8c-bafb-75673b1e8803.pdf,2024-10-10,Yamaha 93306-30605-00 Bearing(4Gb); New # 9330...,Celulares e Smartphones,Yamaha,0,0,0,0,0,1
733,https://www.amazon.com.br/Zein-7899658422059-C...,e4f6ceb0-5b05-4743-a9a3-1aae005683c1.pdf,2024-10-02,CELULAR INTERATIVO TOUCH,Celulares e Smartphones,Zein,7899658422059,0,7899658422059,0,1,1
734,https://www.amazon.com.br/Zenfone-ZS672KS-Desb...,32391894-6dbe-45d5-ac5f-58342af2c989.pdf,2024-10-02,Asus Zenfone 8 Flip ZS672KS 5G Dual 256GB 8GB ...,Celulares e Smartphones,Asus,ASUS_AI2201_C | USB-C,111522203109,0,1,0,1
735,https://www.amazon.com.br/reparo-flexível-comp...,fe38017b-d71e-4d18-8df1-6d8552280850.pdf,2024-10-02,Peças de reparo de cabo flexível compatíveis c...,Celulares e Smartphones,Riihoo,flex-0600 | flex-0600,0,0,0,0,1


In [14]:
df_anuncios_limpos = pd.read_excel('../datasets/inspecao_ecommerce/avaliacao_regulatron/anuncios_para_limpar.xlsx')
df_anuncios_limpos

,#,url,screenshot,coleta,nome,tipo_subcategoria,fabricante,modelo,certificado,ean_gtin,certificado_anuncio,ean_gtin_anuncio,ocorrencias,cor
0,460,https://www.amazon.com.br/Smartphone-Multi-Câm...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,NaN,Celulares e Smartphones,Multilaser,P9173 | F 2,101462203111,0,1,0,4,1
1,461,https://www.amazon.com.br/Smartphone-Multi-Câm...,cf886743-b845-4c89-a8e6-4b378b453248.pdf,2024-10-02,"Smartphone Multi F 2 4G 32GB Tela de 5,5 pol. ...",Celulares e Smartphones,Multilaser,S173 | Multi F2,0,0,0,0,4,1
2,462,https://www.amazon.com.br/Smartphone-Multi-Câm...,cee65ba4-ace0-4fc7-8f28-219a1d2b568e.pdf,2024-10-10,"Smartphone Multi F 2 4G 32GB Tela de 5,5 pol. ...",Celulares e Smartphones,Multilaser,S173 | F 2,101462203111,0,1,0,4,1
3,463,https://www.amazon.com.br/Smartphone-Multi-Câm...,ae2e22de-dfb7-4941-9e16-5b4888dbe350.pdf,2024-10-22,"Smartphone Multi F 2 4G 32GB Tela de 5,5 pol. ...",Celulares e Smartphones,Multilaser,F 2,101462203111,0,1,0,4,1
4,504,https://www.amazon.com.br/Smartphone-Realme-C5...,https://anatel365.sharepoint.com/:b:/r/sites/D...,2024-07-08,NaN,Celulares e Smartphones,Realme,RMX3760 | Realme C53,060522313366,0,1,0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,922,https://www.amazon.com.br/Xiaomi-Smartphone-Me...,550dcd0d-500f-4b6d-b76d-f60a6f3a3da3.pdf,2024-10-02,"Xiaomi POCOPHONE X5 5G (6+128GB, Azul)",Celulares e Smartphones,Xiaomi,22111317PG | Redmi Note 12 5G | 22111317PG,217652209185,0,1,0,1,1
865,919,https://www.amazon.com.br/Xiaomi-SMARTPHONE-RE...,a08f941e-e12f-44f5-85cd-695fc66670bc.pdf,2024-10-22,Xiaomi SMARTPHONE REDMI NOTE 12 8GB/256GB DUAL...,Celulares e Smartphones,Xiaomi,0,32309185,0,1,0,1,2
866,923,https://www.amazon.com.br/Yamaha-93306-30605-0...,154c1059-db5e-4d8c-bafb-75673b1e8803.pdf,2024-10-10,Yamaha 93306-30605-00 Bearing(4Gb); New # 9330...,Celulares e Smartphones,Yamaha,0,0,0,0,0,1,1
867,924,https://www.amazon.com.br/Zein-7899658422059-C...,e4f6ceb0-5b05-4743-a9a3-1aae005683c1.pdf,2024-10-02,CELULAR INTERATIVO TOUCH,Celulares e Smartphones,Zein,7899658422059,0,7899658422059,0,1,1,2


In [20]:
df_resumo_todos = df_anuncios_limpos.groupby('coleta').agg({'url': 'count', 'certificado_anuncio': 'sum', 'ean_gtin_anuncio':'sum'})
df_resumo_todos['%_certificado'] = df_resumo_todos['certificado_anuncio']/df_resumo_todos['url']
df_resumo_todos['%_ean_gtin'] = df_resumo_todos['ean_gtin_anuncio']/df_resumo_todos['url']

df_resumo_todos

,url,certificado_anuncio,ean_gtin_anuncio,%_certificado,%_ean_gtin
coleta,,,,,
2024-07-08,249,249,1,1.000000,0.004016
2024-10-02,162,76,1,0.469136,0.006173
2024-10-10,199,193,1,0.969849,0.005025
2024-10-22,259,259,1,1.000000,0.003861


In [21]:
df_resumo_mais_que_2 = df_anuncios_limpos[df_anuncios_limpos['ocorrencias']>2].groupby('coleta').agg({'url': 'count', 'certificado_anuncio': 'sum', 'ean_gtin_anuncio':'sum'})
df_resumo_mais_que_2['%_certificado'] = df_resumo_mais_que_2['certificado_anuncio']/df_resumo_mais_que_2['url']
df_resumo_mais_que_2['%_ean_gtin'] = df_resumo_mais_que_2['ean_gtin_anuncio']/df_resumo_mais_que_2['url']

df_resumo_mais_que_2

,url,certificado_anuncio,ean_gtin_anuncio,%_certificado,%_ean_gtin
coleta,,,,,
2024-07-08,68,68,0,1.000000,0.0
2024-10-02,73,26,0,0.356164,0.0
2024-10-10,75,75,0,1.000000,0.0
2024-10-22,96,96,0,1.000000,0.0


In [22]:
df_resumo_4 = df_anuncios_limpos[df_anuncios_limpos['ocorrencias']==4].groupby('coleta').agg({'url': 'count', 'certificado_anuncio': 'sum', 'ean_gtin_anuncio':'sum'})
df_resumo_4['%_certificado'] = df_resumo_4['certificado_anuncio']/df_resumo_4['url']
df_resumo_4['%_ean_gtin'] = df_resumo_4['ean_gtin_anuncio']/df_resumo_4['url']

df_resumo_4

,url,certificado_anuncio,ean_gtin_anuncio,%_certificado,%_ean_gtin
coleta,,,,,
2024-07-08,12,12,0,1.000000,0.0
2024-10-02,12,4,0,0.333333,0.0
2024-10-10,12,12,0,1.000000,0.0
2024-10-22,12,12,0,1.000000,0.0


In [44]:
df_anuncios_regulatron.pivot_table(index='url_count', columns='data', values='nome', aggfunc='count').fillna(0).astype(int)

data,2024-10-02,2024-10-03,2024-10-10,2024-10-11,2024-10-22
url_count,,,,,
1,3,7,0,18,0
2,12,16,2,15,15
3,19,5,1,10,13
4,4,1,1,2,0
